# CIFAR-10 Notebook Backup

Este notebook muestra el flujo completo de entrenamiento de un modelo para clasificar imágenes del dataset CIFAR-10. Incluye:
- Carga y preprocesamiento de datos.
- Aumento de datos.
- Entrenamiento del modelo.
- Evaluación y visualizaciones avanzadas.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np

#

## Carga y preprocesamiento de datos

In [ ]:
def load_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = y_train.flatten(), y_test.flatten()
    return (x_train, y_train), (x_test, y_test)

def create_datasets(x_train, y_train, x_test, y_test, batch_size=32):
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size).shuffle(10000)
    val_dataset = train_dataset.take(int(0.2 * len(train_dataset)))
    train_dataset = train_dataset.skip(int(0.2 * len(train_dataset)))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    return train_dataset, val_dataset, test_dataset

(x_train, y_train), (x_test, y_test) = load_data()
train_dataset, val_dataset, test_dataset = create_datasets(x_train, y_train, x_test, y_test)

# 

### Visualización de imágenes del dataset

In [ ]:
class_names = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_train[i])
    plt.title(class_names[y_train[i]])
    plt.axis('off')
plt.show()

# 

## Aumento de datos

In [ ]:
def augment_data(dataset):
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
    ])
    dataset = dataset.map(lambda x, y: (tf.cast(x, tf.float32), y))
    augmented_dataset = dataset.map(lambda x, y: (data_augmentation(x), y))
    return dataset.concatenate(augmented_dataset)

train_dataset = augment_data(train_dataset)

# 

## Definición del modelo

In [ ]:
def create_model():
    model = keras.models.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3)),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.25),
        keras.layers.Conv2D(64, (3, 3)),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.002),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_model()

# 

## Entrenamiento del modelo

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    callbacks=[early_stopping]
)

# 

## Visualización de resultados

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.show()

# 

## Precisión del modelo

In [ ]:
#evaluate


test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.2f}, Test loss: {test_loss:.2f}")

#

## Predicciones en imágenes

In [ ]:
predictions = model.predict(x_test[:9])
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_test[i])
    plt.title(f"Pred: {class_names[np.argmax(predictions[i])]}")
    plt.axis('off')
plt.show()